In [ ]:
import cv2
import numpy as np
import glob, os
import natsort
import re
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
import keras

from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

from keras import models, layers


np.random.seed(77)

In [ ]:
# to get the files in proper order
def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)
# defining the size of the image
SIZE = 100
high_img = []
path = '/content/drive/MyDrive/GAN/Super_resolution/Raw Data/high_res'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):    
    if i == '855.jpg':
        break
    else:    
        img = cv2.imread(path + '/'+i,1)
        # open cv reads images in BGR format so we have to convert it to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #resizing image
        img = cv2.resize(img, (SIZE, SIZE))
        img = img.astype('float32') / 255.0

        high_img.append(img_to_array(img))


low_img = []
path = '/content/drive/MyDrive/GAN/Super_resolution/Raw Data/low_res'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):
     if i == '855.jpg':
        break
     else: 
        img = cv2.imread(path + '/'+i,1)

        #resizing image
        img = cv2.resize(img, (SIZE, SIZE))
        img = img.astype('float32') / 255.0

        low_img.append(img_to_array(img))

100%|██████████| 855/855 [00:05<00:00, 150.90it/s]


In [ ]:
from tensorflow.keras import losses, metrics, optimizers
from sklearn.model_selection import train_test_split

# high_img, low_img
x_train, x_test, y_train, y_test = train_test_split(low_img, high_img)

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
x_train.shape

(641, 100, 100, 3)

In [ ]:
def create_generator():
    generator=models.Sequential()
    generator.add(layers.Dense(units=256,input_dim=10000))
    generator.add(layers.LeakyReLU(0.2))
    
    generator.add(layers.Dense(units=512))
    generator.add(layers.LeakyReLU(0.2))
    
    generator.add(layers.Dense(units=1024))
    generator.add(layers.LeakyReLU(0.2))
    
    generator.add(layers.Dense(units=10000, activation='tanh'))
    
    generator.compile(loss='categorical_crossentropy', optimizer='adam')
    return generator

In [ ]:
def create_discriminator():
    discriminator=models.Sequential()
    discriminator.add(layers.Dense(units=1024,input_dim=10000))
    discriminator.add(layers.LeakyReLU(0.2))
    discriminator.add(layers.Dropout(0.3))
    
    discriminator.add(layers.Dense(units=512))
    discriminator.add(layers.LeakyReLU(0.2))
    discriminator.add(layers.Dropout(0.3))
       
    discriminator.add(layers.Dense(units=256))
    discriminator.add(layers.LeakyReLU(0.2))
    
    discriminator.add(layers.Dense(units=3, activation='sigmoid'))
    
    discriminator.compile(loss='categorical_crossentropy', optimizer='adam')
    return discriminator

In [ ]:
def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = models.Input(shape=10000)
    x = generator(gan_input)
    gan_output= discriminator(x)
    gan= keras.models.Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

def plot_generated_images(epoch, generator, examples=100, dim=(10,10), figsize=(10,10)):
    noise= np.random.normal(loc=0, scale=1, size=[examples, 100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(100,256,256)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image %d.png' %epoch)

In [ ]:
image_batch =x_train[np.random.randint(low=0,high=256,size=30)]
print(image_batch.shape)

(30, 100, 100, 3)


In [ ]:
def training(epochs=1, batch_size=128):

    batch_count = x_train.shape[0] // batch_size
    print(batch_count)
    
    # Creating GAN
    generator = create_generator()
    discriminator = create_discriminator()
    gan = create_gan(discriminator, generator)
    
    """## structure of model
    from keras.utils import plot_model
    plot_model(generator, show_shapes=True, to_file='generator.png')
    
    plot_model(discriminator, show_shapes=True, to_file='discriminator.png')
    
    plot_model(gan, show_shapes=True, to_file='gan.png')"""
    
    for e in range(1,epochs+1):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_size)):
        # Generate random noise as an input to initialize the generator
            noise= np.random.normal(0, 1, [batch_size, 10000])
            
            # Generate fake MNIST images from noised input
            generated_images = generator.predict(noise)
            print(generated_images.shape)
            
            # Get a random set of  real images
            image_batch =x_train[np.random.randint(low=0,high=256,size=batch_size)]
            print(image_batch.shape)
            
            # Construct different batches of real and fake data
            X= np.concatenate([image_batch, generated_images])
            
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=0.9
            
            # Pretrain discriminator on  fake and real data before starting the gan. 
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            # Tricking the noised input of the Generator as real data
            y_gen = np.ones(batch_size)
            
            # During the training of gan, the weights of discriminator should be fixed. 
            # We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            # Training  the GAN by alternating the training of the Discriminator and training the chained GAN model with Discriminator's weights freezed.
            gan.train_on_batch(noise, y_gen)
            
        if e == 1 or e % 20 == 0:
           plot_generated_images(e, generator)

training(400,128)

5
Epoch 1


  0%|          | 0/128 [00:00<?, ?it/s]

(128, 10000)
(128, 100, 100, 3)


ValueError: ignored